# Hyperparameter Tuning using HyperDrive

In [ ]:
from azureml.core import Dataset, Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

import os
import shutil
import joblib

## Dataset

In [ ]:
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="hyperdrive-experiment")
dataset = Dataset.get_by_name(ws, name='heart-disease-kaggle')

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

## Config Compute Cluster

In [ ]:
cpu_cluster = "cpu-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D12_V2',
                                                           max_nodes=5)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


## Hyperdrive Configuration

In [ ]:
# Parameter sampler
ps = RandomParameterSampling({
        "--C": uniform(0.1,1),
        "--max_iter": choice(50, 100, 150, 200)
    })

# Policy
policy = BanditPolicy(evaluation_interval = 3, slack_factor = 0.1, delay_evaluation = 3)

# SKLearn estimator for use with train.py
est = SKLearn(source_directory='.', 
              entry_script='train.py',
              compute_target=cpu_cluster)

# HyperDriveConfig using the hyperparameter sampler, policy and estimator.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy= policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=60,
                                max_concurrent_runs=4)

In [ ]:
# Submitting the hyperdrive run to the experiment
hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)

## Run Details

In [ ]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run ID: ', best_run.id)
print('\n Metrics: ', best_run_metrics)
print('\n Parameters: ', parameter_values)

In [ ]:
best_run

In [ ]:
best_run.download_file('/outputs/model.joblib', 'hyperdrive_model.joblib')

In [ ]:
#Register the best model
best_model_registered = best_run.register_model(model_name = 'best_hyperdrive_model', 
                                                model_path='outputs/model.joblib',
                                                description='best hyperdrive model')